<a href="https://colab.research.google.com/github/he16946/Data_Labs/blob/master/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv', delimiter=',')

df = df.drop(df.columns[0], axis = 1)
df.shape

array = df.values

In [51]:
#example of the returned array when using np.where
x = np.array([[1, 2, 3], [4, 5, 6]], np.int32)
np.where(x!=1)

(array([0, 0, 1, 1, 1]), array([1, 2, 0, 1, 2]))

In [58]:
#creating validation values
def replace(array):

  #make a copy of data
  new_data = array.copy() 

  #return every non-99 index in the array
  rated = np.where(array!=99) 

  #how many values are non-99
  n_rated = len(rated[0]) 

  #returns 2 arrays of indexs of a random 10% sample of non-99s
  idx = np.random.choice(n_rated, size=int(0.1*n_rated), replace = False) 

  #replaces the value at index of the random 10% and replaces them with 99 in the non-99 array
  new_data[rated[0][idx], rated[1][idx]] = 99

  #returns new array with a random 10% of 99s
  return new_data, (rated[0][idx], rated[1][idx])

new_array, idx = replace(array)

#original array value (where idx is an array of indexes)
array[idx[0][0], idx[1][0]] 

#new array value (where the idx index is the same position as in the original)
new_array[idx[0][0], idx[1][0]]


(24982, 100)

In [0]:
n_latent_factors = 2

# Initialise as random values
latent_user_preferences = np.random.random((array.shape[0], n_latent_factors))
latent_item_features = np.random.random((array.shape[1], n_latent_factors))

In [0]:
# user_id (0,24982) number of row
#itedm_id (0,100) number of column

def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

def train(user_id, item_id, rating, alpha=0.0001):
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - return_rating(user_id, item_id)
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=300000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = new_array[user_id][item_id]
                if rating!= 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10000) == 0:
            print(mse)


sgd(10)

In [73]:
def validation_error():
  error = []
  for i in range(0,24982):
    for j in range(0,100):
      val_error =  new_array[idx[0][i], idx[1][j]] - predict_rating(i, j)
      error.append(val_error)
  mse = (np.array(error) ** 2).mean()
  print(mse)   

validation_error()

1176.632320030599
